In [1]:
import numpy as np
import pandas as pd 
import os 
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# Get the current directory of the script
current_dir = os.path.dirname(os.path.abspath('Titantic_Predictor\train.csv'))

# Construct the relative path to the CSV file
csv_file = os.path.join(current_dir, '..', 'Titantic_Predictor', 'train.csv')

# Read the CSV file using pandas
df = pd.read_csv(csv_file)

In [ ]:
age = df["Age"]
survived = df["Survived"]

df.head()
plt.scatter(df["Age"], df["Survived"])
plt.figure(figsize=(10, 10))
plt.show()